In [1]:
from model import WhisperSegmenter, WhisperSegmenterFast
import librosa
import pandas as pd
import numpy as np
from tqdm import tqdm
from copy import deepcopy
from train import validate
from datautils import get_audio_and_label_paths
import os

In [2]:
segmenter = WhisperSegmenter(  model_path = "nccratliri/whisperseg-zebra-finch", 
                        device = "cuda")

In [3]:
fname_list = sorted( [fname for fname in os.listdir("data/dataset/zebseg/") if fname != "all_birds"] )

In [4]:
for fname in fname_list:
    audio_paths, label_paths = get_audio_and_label_paths("data/dataset/zebseg/%s/test/"%(fname))

    audio_list, label_list = [], []
    for audio_path, label_path in zip(audio_paths, label_paths):
        audio, _ = librosa.load( audio_path, sr = 16000 )
        label_df = pd.read_csv( label_path )
        label = {
            "onset":list(label_df["onset"]),
            "offset":list(label_df["offset"]),
            "cluster":list(label_df["cluster"]) 
        }
        audio_list.append(audio)
        label_list.append(label)
    
    
    res = validate( audio_list, label_list, segmenter, sr = 16000, tolerance = 0.02, num_trials = 3 , min_segment_length = 0.02,
              voting_time_step = 1.0, voting_precision = 0.001, batch_size = 8, max_length = 400, target_cluster = None )
    print(fname+":\t", res)
    

100%|██████████████████████████████████████████████████████████| 1/1 [00:04<00:00,  4.09s/it]


R3406_035:	 (1.0, 1.0, 1.0, 17, 17, 17)


100%|██████████████████████████████████████████████████████████| 5/5 [00:07<00:00,  1.40s/it]


R3406_045:	 (1.0, 1.0, 1.0, 4, 4, 4)


100%|████████████████████████████████████████████████████████| 19/19 [00:29<00:00,  1.56s/it]


...

g19o10:	 (1.0, 1.0, 1.0, 194, 194, 194)


100%|████████████████████████████████████████████████████████| 10/10 [00:21<00:00,  2.16s/it]


g19o3:	 (1.0, 1.0, 1.0, 215, 215, 215)


100%|████████████████████████████████████████████████████████| 82/82 [03:53<00:00,  2.84s/it]

g4p5:	 (0.9987903225806452, 0.9975835682641966, 0.998186580697159, 2477, 2480, 2483)
